In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Longevity"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Longevity/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Longevity/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Longevity/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Longevity/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# 1. List all subdirectories in the TCGA root directory
subdirectories = os.listdir(tcga_root_dir)
print(f"Available TCGA subdirectories: {subdirectories}")

# The target trait is Longevity
target_trait = trait.lower()  # "longevity"

# For longevity, we should look for datasets with good survival/aging data
# rather than matching the term directly to a cancer type
print(f"Searching for datasets suitable for {target_trait} analysis...")

# Since longevity relates to survival time and patient outcomes, 
# we'll select a cancer type with typically good survival data coverage
# Choosing TCGA_Breast_Cancer_(BRCA) as it typically has a large cohort with varied survival outcomes
selected_dir = "TCGA_Breast_Cancer_(BRCA)"

# Verify the directory exists
if selected_dir in subdirectories:
    print(f"Selected directory: {selected_dir} for longevity analysis")
    
    # 2. Get the clinical and genetic data file paths
    cohort_dir = os.path.join(tcga_root_dir, selected_dir)
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic file: {os.path.basename(genetic_file_path)}")
    
    # 3. Load the data files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # 4. Print clinical data columns for inspection
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Print basic information about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")
    
    # Check if the clinical data contains survival-related columns
    survival_columns = [col for col in clinical_df.columns if any(term in col.lower() for term in 
                       ['survival', 'death', 'follow', 'vital', 'days_to'])]
    print("\nSurvival-related columns found:")
    print(survival_columns)
    
    # Check if we have both gene and survival data
    is_gene_available = genetic_df.shape[0] > 0
    is_trait_available = clinical_df.shape[0] > 0 and len(survival_columns) > 0
    
else:
    print(f"Directory {selected_dir} not found. Checking for alternative datasets...")
    # Alternative approach: search for a dataset with good survival information
    is_gene_available = False
    is_trait_available = False
    
    for subdir in subdirectories:
        if os.path.isdir(os.path.join(tcga_root_dir, subdir)) and not subdir.startswith('.'):
            try:
                cohort_dir = os.path.join(tcga_root_dir, subdir)
                clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
                
                # Quick check of clinical data
                clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
                genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
                
                # Look for survival columns
                survival_columns = [col for col in clinical_df.columns if any(term in col.lower() for term in 
                                   ['survival', 'death', 'follow', 'vital', 'days_to'])]
                
                if len(survival_columns) > 0 and clinical_df.shape[0] > 100 and genetic_df.shape[0] > 0:
                    print(f"Selected alternative directory: {subdir}")
                    print(f"Clinical file: {os.path.basename(clinical_file_path)}")
                    print(f"Genetic file: {os.path.basename(genetic_file_path)}")
                    print(f"\nClinical data shape: {clinical_df.shape}")
                    print(f"Genetic data shape: {genetic_df.shape}")
                    print("\nSurvival-related columns found:")
                    print(survival_columns)
                    
                    is_gene_available = True
                    is_trait_available = True
                    break
                    
            except Exception as e:
                print(f"Error processing {subdir}: {e}")
                continue

# Record the data availability
validate_and_save_cohort_info(
    is_final=False,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Exit if no suitable directory was found
if not is_gene_available or not is_trait_available:
    print("Skipping this trait as no suitable survival data was found in TCGA.")


Available TCGA subdirectories: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Canc


Clinical data columns:
['AJCC_Stage_nature2012', 'Age_at_Initial_Pathologic_Diagnosis_nature2012', 'CN_Clusters_nature2012', 'Converted_Stage_nature2012', 'Days_to_Date_of_Last_Contact_nature2012', 'Days_to_date_of_Death_nature2012', 'ER_Status_nature2012', 'Gender_nature2012', 'HER2_Final_Status_nature2012', 'Integrated_Clusters_no_exp__nature2012', 'Integrated_Clusters_unsup_exp__nature2012', 'Integrated_Clusters_with_PAM50__nature2012', 'Metastasis_Coded_nature2012', 'Metastasis_nature2012', 'Node_Coded_nature2012', 'Node_nature2012', 'OS_Time_nature2012', 'OS_event_nature2012', 'PAM50Call_RNAseq', 'PAM50_mRNA_nature2012', 'PR_Status_nature2012', 'RPPA_Clusters_nature2012', 'SigClust_Intrinsic_mRNA_nature2012', 'SigClust_Unsupervised_mRNA_nature2012', 'Survival_Data_Form_nature2012', 'Tumor_T1_Coded_nature2012', 'Tumor_nature2012', 'Vital_Status_nature2012', '_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_BRCA', '_PANCAN_DNAMethyl_PANCA

### Step 2: Find Candidate Demographic Features

In [3]:
# Find candidate age columns
candidate_age_cols = [
    "Age_at_Initial_Pathologic_Diagnosis_nature2012",
    "age_at_initial_pathologic_diagnosis",
    "days_to_birth"  # Can be used to calculate age
]

# Find candidate gender columns
candidate_gender_cols = [
    "Gender_nature2012",
    "gender"
]

# Preview the selected columns
# First, get the clinical file path
cohort_dir = os.path.join(tcga_root_dir, "TCGA_Breast_Cancer_(BRCA)")
clinical_file_path, _ = tcga_get_relevant_filepaths(cohort_dir)

# Load the clinical data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Extract candidate age columns
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()

# Extract candidate gender columns
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()

# Display previews
print("Age columns preview:")
print(age_preview)
print("\nGender columns preview:")
print(gender_preview)


Age columns preview:
{'Age_at_Initial_Pathologic_Diagnosis_nature2012': [nan, nan, nan, nan, nan], 'age_at_initial_pathologic_diagnosis': [55.0, 50.0, 62.0, 52.0, 50.0], 'days_to_birth': [-20211.0, -18538.0, -22848.0, -19074.0, -18371.0]}

Gender columns preview:
{'Gender_nature2012': [nan, nan, nan, nan, nan], 'gender': ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']}


### Step 3: Select Demographic Features

In [4]:
# Analyzing available demographic columns

# For age columns
age_columns = {'Age_at_Initial_Pathologic_Diagnosis_nature2012': [float('nan'), float('nan'), float('nan'), float('nan'), float('nan')], 
               'age_at_initial_pathologic_diagnosis': [55.0, 50.0, 62.0, 52.0, 50.0], 
               'days_to_birth': [-20211.0, -18538.0, -22848.0, -19074.0, -18371.0]}

# For gender columns
gender_columns = {'Gender_nature2012': [float('nan'), float('nan'), float('nan'), float('nan'), float('nan')], 
                 'gender': ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']}

# Select age column - choose the one with non-NaN values
age_col = None
for col_name, values in age_columns.items():
    # Check if the column has non-NaN values
    if not all(pd.isna(val) for val in values):
        # Prefer direct age values over days_to_birth
        if 'age' in col_name.lower():
            age_col = col_name
            break

# If no age column with 'age' in name found, consider days_to_birth
if age_col is None and 'days_to_birth' in age_columns and not all(pd.isna(val) for val in age_columns['days_to_birth']):
    age_col = 'days_to_birth'

# Select gender column - choose the one with non-NaN values
gender_col = None
for col_name, values in gender_columns.items():
    # Check if the column has non-NaN values
    if not all(pd.isna(val) for val in values):
        gender_col = col_name
        break

# Print the selected columns
print(f"Selected age column: {age_col}")
print(f"Selected gender column: {gender_col}")


Selected age column: age_at_initial_pathologic_diagnosis
Selected gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Create clinical features dataframe with trait (Canavan Disease) using patient IDs
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Step 2: Normalize gene symbols in the gene expression data
# The gene symbols in TCGA genetic data are already standardized, but we'll normalize them for consistency
normalized_gene_df = normalize_gene_symbols_in_index(genetic_df)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")
print(f"Gene expression data shape after normalization: {normalized_gene_df.shape}")

# Step 3: Link clinical and genetic data
# Transpose genetic data to have samples as rows and genes as columns
genetic_df_t = normalized_gene_df.T
# Save the clinical data for reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_features.shape}")

# Verify common indices between clinical and genetic data
clinical_indices = set(clinical_features.index)
genetic_indices = set(genetic_df_t.index)
common_indices = clinical_indices.intersection(genetic_indices)
print(f"Number of samples in clinical data: {len(clinical_indices)}")
print(f"Number of samples in genetic data: {len(genetic_indices)}")
print(f"Number of common samples: {len(common_indices)}")

# Link the data by using the common indices
linked_data = pd.concat([clinical_features.loc[list(common_indices)], genetic_df_t.loc[list(common_indices)]], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# Step 4: Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Step 5: Determine whether the trait and demographic features are severely biased
trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait=trait)

# Step 6: Conduct final quality validation and save information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=linked_data,
    note=f"Dataset contains TCGA glioma and brain tumor samples with gene expression and clinical information for {trait}."
)

# Step 7: Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable based on validation criteria. Data not saved.")

print("Preprocessing completed.")

Normalized gene expression data saved to ../../output/preprocess/Longevity/gene_data/TCGA.csv
Gene expression data shape after normalization: (19848, 1218)
Clinical data saved to ../../output/preprocess/Longevity/clinical_data/TCGA.csv
Clinical data shape: (1247, 3)
Number of samples in clinical data: 1247
Number of samples in genetic data: 1218
Number of common samples: 1218
Linked data shape: (1218, 19851)


Data shape after handling missing values: (1218, 19851)
For the feature 'Longevity', the least common label is '0' with 114 occurrences. This represents 9.36% of the dataset.
The distribution of the feature 'Longevity' in this dataset is fine.

Quartiles for 'Age':
  25%: 48.0
  50% (Median): 58.0
  75%: 67.0
Min: 26.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 13 occurrences. This represents 1.07% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Longevity/TCGA.csv
Preprocessing completed.
